# Stochastic block model analysis using `graph-tool`

**Warning:** This notebook can only run on a Linux (or Mac OS) as the `graph-tool` module is not available on Windows

In [105]:
import graph_tool.all as gt
import sys
sys.path.append(r"../")

from graph import Network

In [106]:
# Loading network from file

filename = r"../out/last/RTH_E3O2P21R12T17_Empty_L10_C750_S50_T0_444675_distrib.h5"
net = Network()
net.reload_with_hdf5(filename)

In [107]:
# Network to graph tool graph

g = gt.Graph()
etrust = g.new_edge_property("double")
elink = g.new_edge_property("int")
g.edge_properties["trust"] = etrust
g.edge_properties["link"] = elink
g.add_edge_list(net.get_edge_list(), eprops=[g.ep.trust, g.ep.link])

ph = g.new_vertex_property("string", vals=net.phenotypes_table)
g.vertex_properties["Phenotype"] = ph

In [112]:
state = gt.minimize_nested_blockmodel_dl(g)

Tab10 colors:
- tab:blue : #1f77b4
- tab:orange : #ff7f0e
- tab:green : #2ca02c
- tab:red : #d62728
- tab:purple : #9467bd

In [113]:
# Setting color based on phenotype
color_dict = {
    "Envious": 'tab:blue',
    "Optimist": 'tab:orange',
    "Pessimist": 'tab:green',
    "Random": 'tab:red',
    "Trustful": 'tab:purple'
}
vcolors = []
ph_table = net.phenotypes_table
for i in range(len(ph_table)):
    vcolors.append(color_dict[ph_table[i]])
vcolorp = g.new_vertex_property("string", vals=vcolors)

In [114]:
# Setting edges appearence
esizes = []
edge_list = net.get_edge_list()
for i, j, t, l in edge_list:
    esizes.append(float(l))
esizep = g.new_edge_property('float', vals=esizes)

In [116]:
img = state.draw(
            output=r"../images/sbm_rth.png", 
            vprops = {
                "size": 10,
                "halo": True,
                "halo_color": vcolorp,
                "halo_size": 2, 
                "color": [0, 0, 0, 0.5]
            },
            eprops= {
                "pen_width": esizep
            }
)